# Chapter 5. 函数 II
## Part 1 函数进阶

在本部分，你会了解到：
- 更复杂的递归
- 迭代器和生成器
- 函数式编程初步

在进入本章的学习之前，默认你已经认真看了Chapter 4，学会了自己定义函数。

Part 1将在上一章的基础之上介绍一些进阶内容，这些内容对于初学者而言略有超纲，你未必需要掌握，但偶尔秀几次会让代码格调满满。注意，**略过Part 1不会对你写代码产生任何影响，但会影响你理解别人写的代码，尤其是出自大神之手的代码**。

预告一下，Part 2中关于第三方模块的知识，在将来使用Python的过程中绕不开，需要额外关注。

_____
## 5.1 递归函数进阶

在Chapter 4，我们以阶乘和斐波那契数列为例，介绍了最简单的递归函数。我们知道了，递归，无非就是把问题化大为小。本章将介绍一个更复杂的情形：用递归思想解决**汉诺塔**问题。

汉诺塔（the tower of Hanoi）起源于印度的古老传说。大梵天创造世界的时候做了三根金刚石柱子，在一根柱子上从下往上按照大小顺序摞着64片黄金圆盘。大梵天命令婆罗门把圆盘从下面开始按大小顺序重新摆放在另一根柱子上。并且规定，在小圆盘上不能放大圆盘，在三根柱子之间一次只能移动一个圆盘。

事实上，就跟在国际象棋棋盘上摆大米粒的传说一样，婆罗门一辈子也不可能完成这项任务。后来，这个传说发展成为一项益智游戏，圆盘的数量呢，一般在3到8之间。见下图，游戏的规则，就是把盘子从柱A，全部转移到B或者C，并且在转移的过程中，大盘子自始至终只能放在小盘子之下。

![hanoi](pics/hanoi.jpg)

为了更好地理解这个问题，你可以尝试在线玩[汉诺塔小游戏](http://www.7k7k.com/swf/335.htm)。

现在，我们要设计一个函数Towers()。这个函数是打印出任意阶数汉诺塔游戏的最快操作步骤。

比如，对于3阶汉诺塔，我们要把圆盘从A移到C，那Towers()函数的输出应该是

```
move from A to C
move from A to B
move from C to B
move from A to C
move from B to A
move from B to C
move from A to C
```

如何用递归思想实现该函数？显然，需要把问题化大为小。为了简化，让我们考虑三阶汉诺塔由A移动到C的问题。

要完成这个游戏，我们必然要把最大的圆盘移到目标柱C上。而在实现这步操作之前，必然满足A柱只剩下最大圆盘，而另外两个圆盘在B柱。若把这两个圆盘看成整体，那我们一开始的目的是把它们移动到柱B，而这是一个二阶汉诺塔问题。

把最大圆盘移动到C柱以后，我们又面临了一个二阶汉诺塔问题，即把较小的两个圆盘从柱B移动到C。而这个问题又可以继续往下分解...

三阶汉诺塔游戏可以分解为：先把较小的两个圆盘移到柱B，再把最大的圆盘移到柱C，再把较小的两个圆盘由柱B移动到柱C。

依此类推，N阶汉诺塔可以看成是先完成N-1阶汉诺塔游戏，再把最大的圆盘移动到目标柱（1阶汉诺塔），之后接着玩一轮N-1阶汉诺塔游戏。

这启示我们，如果用递归写Towers()函数，在遇到1阶汉诺塔问题时，可以直接打印操作步骤，若遇到N阶汉诺塔问题，则分解为N-1阶、1阶、N-1阶三个子问题。



In [ ]:
def printMove(fr, to):
    print('move from ' + str(fr) + ' to '+ str(to))

def Towers(n, fr, to, spare):
    '''
    n：圆盘数量
    fr: 起始柱
    to: 目标柱
    spare: 空柱
    '''
    if n == 1:
        printMove(fr, to)  #如果只剩一个圆盘，运行printMove()函数
    else:
        Towers(n-1, fr, spare, to)  #把较小的n-1个圆盘移动到空柱
        Towers(1, fr, to, spare)  #把剩下的圆盘移动到目标柱
        Towers(n-1, spare, to, fr) #再把n-1个圆盘移动到目标柱

In [ ]:
Towers(5,'A','B','C')

请对照注释理解该函数。作为补充材料，可以观看课程目录下的Hanoi_MITX6001.mp4，这是MIT计算机系Eric Grimson教授对汉诺塔问题的讲解。

还是那句话，在Python初学者这个阶段，递归并不常用。本教程举的递归函数例子，主要是为了加深读者对函数的理解。

_____
## 5.2 迭代器和生成器

### 5.2.1 迭代器（Iterator）

在Chapter 3介绍迭代时，我们已经碰到了一个概念叫可迭代对象（Iterable）。凡是可作用于for循环的对象都是Iterable类型。之前我们遇到的Iterable有list、tuple、dict、set、str、range对象等等。除此以外，还有一类特殊的Iterable，叫做迭代器（Iterator），它的特点在于能够用next()函数一一提取其中的元素。

迭代器本身是可迭代对象，但反之则未必成立。我们可以通过iter()函数把Iterable转化为Iterator：

In [ ]:
from collections import Iterable,Iterator  #导入Iterable，Iterator两个类，这样就可以利用isinstance()判断一个对象是否属于Iterable或Iterator

In [ ]:
ll = [1,2,3,4,5]

In [ ]:
isinstance(ll, Iterable) #判断ll是否是Iterable。isinstance()函数在Chapter 3的作业里有所涉及。

In [ ]:
isinstance(ll, Iterator) #判断ll是否是Iterator

我们定义了列表ll。使用isinstance()函数，我们发现ll是Iterable，但不是Iterator，于是使用iter()函数将其转化为迭代器：

In [ ]:
ll_iter = iter(ll)

In [ ]:
ll_iter

In [ ]:
print(ll_iter)

In [ ]:
isinstance(ll_iter,Iterable)

In [ ]:
isinstance(ll_iter,Iterator)

现在ll是迭代器了（而迭代器本身又是可迭代对象）。我们不能直接看到迭代器里的元素，也无法用print()函数打印一个迭代器，而需要通过next()函数提取元素值。每调用一次，就顺次返回一个值，直到迭代器内容为空，引发StopIteration错误为止：

In [ ]:
next(ll_iter)

In [ ]:
next(ll_iter)

In [ ]:
next(ll_iter)

In [ ]:
next(ll_iter)

In [ ]:
next(ll_iter)

In [ ]:
next(ll_iter)

迭代器只能从头到尾走一遍，是一个“一次性”的东西。一旦出现StopIteration，意味着再也无法从其中榨取出新元素了。程序出错会中断整个脚本的运行，为了避免出现StopIteration错误，我们可以直接用for循环调取内部元素：

In [ ]:
ll_iter = iter(ll)
for i in ll_iter:
    print(i)

注意，一次性的东西，不可以用两次，所以再次调用for循环时，不会有任何输出结果（这是初学者常踩的坑）：

In [ ]:
for i in ll_iter:
    print(i)

因为是可迭代对象，所以迭代器可以用list()函数转化成列表，用tuple()函数转化成元组，但是注意，转化过一次以后，迭代器就空了：

In [ ]:
ll_iter = iter(ll)
ll_iter

In [ ]:
list(ll_iter)

In [ ]:
tuple(ll_iter)  #输出结果是空元祖

迭代器的设计初衷是为了降低对内存的消耗。打个比方，如果桌子大小一定，假设就一张普通办公桌大小，那一次性把满汉全席端上桌是会出事儿的；一道一道上菜，菜与菜之间有所缓冲，就可以缓解这个问题。后者就是迭代器的思路。

除了iter()以外，Python的其他内置函数也能返回迭代器，Chapter 3介绍的enumerate()函数便是其中之一，在此做简单的回顾。在遍历一个Iterable时，如果我们还要知道取出的元素在iterable内部的序号，则可以使用enumerate()函数：

In [ ]:
for i, value in enumerate(['A', 'B', 'C']):
    print(i, value)

In [ ]:
isinstance(enumerate(['A', 'B', 'C']),Iterator)

另一个内置函数reversed()也返回迭代器。顾名思义，reversed()的功能是把序列（sequence），如list、tuple、range对象等，进行逆向排序：

In [ ]:
for i in reversed(range(1,11)):
    print(i)

In [ ]:
isinstance(reversed(range(1,11)), Iterator)

还有一个内置函数zip(),能把多个序列里的元素按顺序一一粘在一起，返回一个迭代器，你可以从下列代码的输出结果中找到规律：

In [ ]:
for x in zip(['a','b','c'],[1,2,3]):
    print(x)

In [ ]:
isinstance(zip(['a','b','c'],[1,2,3]),Iterator)

In [ ]:
for x in zip(['a','b','c'],[1,2,3],[True,True,False]):
    print(x)

In [ ]:
for x in zip(['a','b','c'],[1,2,3],[True,True,False],['你','我','他']):
    print(x)

显然，zip()函数不限制输入序列的个数，这是否让你想起了Chapter 4介绍的可变参数（arbitrary arguments）？所以也可以用以下方式调用zip()：

In [ ]:
tuple_list = (['a','b','c'],[1,2,3],[True,True,False])
for x in zip(*tuple_list): 
    print(x)

zip()函数遵循木桶原则，当各序列长度不一样时，返回的迭代器长度取决于最短的那个序列：

In [ ]:
for x in zip(['a','b','c','d'],[1,2,3],[True,True]):
    print(x)

像迭代器这样尽量不占用系统资源的属性，还有很多序列型的数据结构都具备。典型的如range()函数返回的对象、dict.items()返回的对象等，在Chapter 3已有提及，此处再次强调。不过注意，因为不是迭代器，它们是不能用next()来提取元素的。

在Python 2版本，有很多函数或方法都是直接返回列表的，但在元素数量多的情况下，这种方式占用内存。为了提升程序执行效率，Python 3优化了这些函数的返回形式。在Python 2.7中range()函数返回的就是列表，而xrange()返回xrange对象，许多进阶教程都推荐用xrange()代替range()。Python 3干脆将xrange()命名为range()，原来的range()被抛弃了。作为参考，可以看这个帖子: [Python 2.7 xrange和range的使用区别](http://blog.csdn.net/humanking7/article/details/45950967)。

### 5.2.2 生成器（Generator）

生成器是快速生成迭代器的一种方式。我们可以通过生成器加深对迭代器的理解。

Chapter 3介绍了一个强有力的工具——列表生成式（List Comprehension）。举例回顾：

In [ ]:
[i*i for i in range(5)]

这个列表生成式，和生成器生成式只有一步之遥，只要把中括号[]换成圆括号()就可以了：

In [ ]:
(i*i for i in range(5))

In [ ]:
g = (i*i for i in range(5))

In [ ]:
isinstance(g, Iterator)

可以看到，将[]改为()后，返回的是一个generator object。生成器是迭代器的子类，自然，我们可以用next()函数查看生成器里的内容，且为了避免抛出StopIteration错误，可以用for循环遍历：

In [ ]:
next(g)

In [ ]:
next(g)

In [ ]:
next(g)

In [ ]:
next(g)

In [ ]:
next(g)

In [ ]:
next(g)

In [ ]:
g = (i*i for i in range(5))
for i in g:
    print(i)

Python有两个内置函数any()和all()可以直接接收以上形式的生成器作为输入：

In [ ]:
any(i for i in [True,False,False,False,False])

In [ ]:
any(i for i in [True,True,True,True,True])

In [ ]:
all(i for i in [True,True,True,True,False])

In [ ]:
all(i for i in [True,True,True,True,True])

以上两个函数很简单，观察函数名、函数输入和输出就能判断它们的作用，在此不做展开。

生成器还可以用类似函数的形式定义，然后像函数一样调用。作为一个例子，我们先看看如何用普通函数打印指定范围的斐波那契数列（关于斐波那契的介绍请参看Chapter 4）：

In [ ]:
def fib(length):
    n, a, b = 0, 0, 1
    while n < length:
        print(b)
        a, b = b, a + b  #不借助临时变量实现值交换
        n += 1

In [ ]:
fib(5)

而我们只需要对以上函数稍作修改就可以定义一个生成器：

In [ ]:
def fib_gen(length):
    n, a, b = 0, 0, 1
    while n < length:
        yield b
        a, b = b, a + b  #不借助临时变量实现值交换
        n += 1

In [ ]:
fib_gen(5)

In [ ]:
fibs = fib_gen(5)

In [ ]:
next(fibs)

In [ ]:
next(fibs)

In [ ]:
next(fibs)

In [ ]:
next(fibs)

In [ ]:
next(fibs)

In [ ]:
next(fibs)

In [ ]:
fibs = fib_gen(5)
for i in fibs:
    print(i)

如果你仔细观察，会发现生成器用yield语句替代了函数里的print语句。yield看上去好像和函数的return语句有类似作用，但实则不然。

在函数中，一旦触发return，就返回函数值，且函数执行完毕，触发的return语句后面即便有别的代码，也不执行了。但在生成器里，yield虽然也触发了返回值，但不代表运行逻辑的完结，而只是暂停。下一次调用next()时，生成器将从暂停的地方恢复执行，以如下生成器为例：

In [ ]:
def gen():
    print('step 1')
    yield 1
    print('step 2')
    yield(3)
    print('step 3')
    yield(5)

In [ ]:
g2 = gen()

In [ ]:
next(g2)

In [ ]:
next(g2)

In [ ]:
next(g2)

In [ ]:
next(g2)

可以看到，gen()不是普通函数，而是generator，在执行过程中，遇到yield就中断，下次又继续执行。执行3次yield后，已经没有yield可以执行了，所以，第4次调用next(g2)就报错。

利用generator，可以生成无穷数列，如全体自然数、全体斐波那契数列等，我们只要一直让循环持续，永不跳出即可。以永续斐波那契数列为例：

In [ ]:
def fib_infinite():
    a, b =  0, 1
    while True:
        yield b
        a, b = b, a + b

In [ ]:
fibs_i = fib_infinite()

In [ ]:
next(fibs_i) #你可以不断按Ctrl + Enter执行这个代码框。

这就是生成器的魅力所在。我们不可能把无穷长的数列装进一个list，即使可以，计算机内存也负荷不了。然而我们可以把生成无穷长数列的规则存起来，封装成一个生成器，用的时候一次蹦一个，细水长流。

生成器、迭代器、可迭代对象、list、set、dict等的关系可以用下图表示：


![](pics/relationships.png)
source: http://nvie.com/posts/iterators-vs-generators/

_____
## 5.3 函数式编程（Functional Programming）
函数式编程，是一种高级的、抽象的编程形式，如果使用得当，能在简化代码的同时增强可读性。本小节除了介绍基本概念以外，还将详细介绍匿名函数、map()、reduce()、filter()、sorted()等强有力的工具。

### 5.3.1 高阶函数

Chapter 2开头部分提到，在Python里，一切皆是对象（object）。自然地，函数也是一种对象。既然是对象，就有名字，函数名就是指向该函数对象的一个变量名。对于这一知识点，Chapter 2、 Chapter 4作业部分都有暗示。

在Chapter 2，我们把1赋值给print，再运行print(123)时提示出错，因为此时print指向了1而不是函数对象。

在Chapter 4作业的task 1，我们定义了一个函数a():

In [ ]:
def a(x):
   '''
   x: int or float.
   '''
   return x + 1

1.5 小题问大家a的数据类型，很显然，a的数据类型是函数：

In [ ]:
a

In [ ]:
type(a)

在Chapter 4作业的task 2，我们定义了一个函数二元一次多项式求值函数a()，但在传入参数后报错：

In [ ]:
def a(a,b,c,x):
    return a*x**2+b*x+c
a,b,c,x = (1,2,3,4)
a(a,b,c,x)

这是因为，定义好a()函数以后，变量a起初指向了这个求值函数，但在之后a被赋值为1，所以不能通过a()召唤原函数。

以上例子告诉我们，变量可以指向函数对象。当然，正如我们可以把字符对象、数值对象、列表对象等赋值给一个变量，一个函数对象本身，也可以赋值给变量：

In [ ]:
abs

In [ ]:
abs(-123)

In [ ]:
f = abs

In [ ]:
f(-123)

在上面这个例子中，我们把系统内置的绝对值函数abs()传递给了变量f，之后，f()就具备了求绝对值的功能。

函数还能作为参数，输入至其他函数，例如在add()这个函数里，参数f接受的是函数：

In [ ]:
def add(x, y, f):
    return f(x) + f(y)

In [ ]:
add(-1,1,abs)

In [ ]:
add(-2.45,5.66,round)

add()的参数有x,y,f，f是函数，在返回函数值时，x和y作为参数分别传入到f()函数里。

像add()这样接受函数对象作为传入参数的特殊函数，称为**高阶函数**。而这种coding style体现的就是函数式编程。

### 5.3.2 匿名函数
匿名函数在Python中的出镜率挺高。应对不同的问题，我们会有针对性地写函数。但是有些函数的实现很简单，在这种情况下，使用匿名函数一行即可搞定。

在Python里，匿名函数的书写语法是 **`lambda 参数（多个参数以逗号隔开）: 利用参数做的运算`**，下面举几个简单的例子。首先是求平方的函数，我们先按照以前的方式定义：

In [ ]:
def square(x):
    return x**2

In [ ]:
square(4)

而用匿名函数，可以把求平方函数表示为：

In [ ]:
lambda x: x**2

运行上面这行代码返回的是一个函数，为了测验这个函数，我们把它传递给某个变量：

In [ ]:
square_lambda = lambda x: x**2 

In [ ]:
square_lambda(4)

从结果上看，显然，匿名函数起到了预想的功能，且只用了一行。我们再看多参数的例子：

In [ ]:
power = lambda x,n : x**n

In [ ]:
power(2,3)

In [ ]:
power(2,4)

上面这个匿名函数实现的功能是求幂，所以有两个参数x和n，用逗号隔开。

把匿名函数传递给某个变量的做法并不普遍，在此处只是为了说明其函数本质。事实上，匿名函数往往被用作参数传入到高阶函数里，这样做的好处是避免为了使用高阶函数而专门定义一个新函数。接下去的小节将介绍常用的四个高阶函数，在其中我们能看到匿名函数的灵活使用。

### 5.3.3 常用高阶函数

Python中最常用的四个高阶函数分别是map()、reduce()、filter()和sorted()。

**map/reduce**

首先介绍map()，它相当于一个“格式刷”，我们先看一下它的函数说明：
```
 |  map(func, *iterables) --> map object
 |  
 |  Make an iterator that computes the function using arguments from
 |  each of the iterables.  Stops when the shortest iterable is exhausted.
```
需要说明的是，在Python 2里，map()是一个函数，但在Python 3，map是一个类（class），而map()是生成map类的途径。关于类的概念我们将在接下来的两章介绍，目前你依旧可以把map()理解成一个普通函数。

我们看它的参数有两部分，第一个位置参数是func，显然，是function的意思。第二个参数\*iterables是可变参数，表示接收可迭代对象，如list、set、str等。再看函数说明，"make an iterator"说明函数本身的返回对象是一个迭代器，"computes the function using arguments from each of the iterables"是说，从传入的iterable里依次取元素，放入函数内部求值。也就是说，最后返回的迭代器，应当包含了所有的函数值。

我们来看一个简单的例子：

In [ ]:
def square(x):
    return x**2

sq = map(square,[1,2,3,4,5])

In [ ]:
type(sq)

In [ ]:
list(sq)

In [ ]:
list(map(square, [1,2,3,4,5]))

在上面这个例子中，我们先定义了求平方的函数square()，之后再把函数名作为参数传至map()函数。生成的sq是一个map对象，因为是迭代器，可以直接用list转化为列表。从结果上看，相当于对原列表[1,2,3,4,5]的每个元素依次进行了求平方操作。所以map()相当于一把格式刷，把传入的函数“刷”给可迭代对象的每一个元素。如果不使用map()，实现相同功能的代码如下：

In [ ]:
l = []
for i in [1,2,3,4,5]:
    l.append(square(i))
l

显然，使用map大大简化了代码，同时也增强了可读性。

请回忆上一小节匿名函数的内容。在使用常用高阶函数时，我们甚至没有必要专门去定义一个函数作为参数，如上例中的square()函数，就可以用匿名函数替代：

In [ ]:
list(map(lambda x: x**2, [1,2,3,4,5]))

匿名函数也是函数，所以可以作为map()的参数。此处**lambda x: x\*\*2**应当看成一个整体。用匿名函数的好处是什么呢？在一些情况下，有些临时性的函数非常简单，而且使用频率很低，如果按def f()的方式去定义它们，显得代码冗余。当然，如果一个函数很复杂，或者后续使用频率挺高，我们就不要强行使用匿名函数了。

map()的孪生兄弟是reduce()，然而，在Python 3，这个函数被打入冷宫，我们需要从functools模块导入它：

In [ ]:
from functools import reduce

In [ ]:
help(reduce)

reduce()的两个参数分别是函数和序列。传入的函数必须接收且只接受两个参数，reduce()可以把这个函数作用在序列上。一开始是取最前面的两个元素作为函数的输入，得到返回值后，又把返回值和序列的下一个元素作为函数输入。最终效果就是：
`reduce(f, [x1, x2, x3, x4]) = f(f(f(x1, x2), x3), x4)`

reduce被打入冷宫也不奇怪，因为它的使用场景较map少了许多，主要用于实现连续加法或连续乘法：

In [ ]:
reduce(lambda x,y : x+y, [1,2,3,4,5,6,7,8,9])

In [ ]:
reduce(lambda x,y : x*y, range(1,6))

上面两行代码只是为了举例，其实普通的连续加法直接用sum()就可以了。在某种程度上，使用reduce()函数并没有那么直观，所以许多程序员宁可用循环替代。

不过，结合使用map/reduce，还是有点威力的。现在，假我们不知道int()，我们要利用map和reduce定义一个函数str_to_int()，把字符型整数转化为数值型。如，把'12345'转化为12345。

因为字符串也是序列，是可迭代对象，可以传入map()。在整数型字符串里，每个位置只有10种可能（'0','1'...到'9'）。我们可以建立一个映射函数，利用字典，把'0'转化为0，'1'转化为1，依此类推。然后再利用map()函数把该映射函数“刷”到目标字符串上，下面以'12345'为例：

In [ ]:
def char2num(s):
    numdict = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
    return numdict[s]

In [ ]:
list(map(char2num, '12345'))

获得这个列表以后，我们再还原成五位数。`12345 = (((1*10+2)*10+3)*10+4)*10+5`，我们可以利用reduce()简化这个嵌套：

In [ ]:
reduce(lambda x,y: x*10+y, [1,2,3,4,5])

将以上步骤打包成str_to_int():

In [ ]:
def str_to_int(s):
    return reduce(lambda x,y: x*10+y, map(char2num,s))

In [ ]:
str_to_int('12345')

注意，map类本身就是可迭代对象，可以直接传入reduce()，没有必要转化为list。

其实用int('12345')就可以完成上述转化，所以以上例子仅供参考。

**过滤器filter**

和map()一样，filter()也接收一个函数和一个可迭代对象。filter()把传入的函数依次作用于可迭代对象的每一个元素，若返回值为真，则保留该元素。最后，将保留的元素以迭代器的形式返回。它的作用，就像一个过滤器。

举一个简单的例子，在一个list中删掉3的倍数，可用filter()表示如下：

In [ ]:
list(filter(lambda x: x % 3 != 0, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]))

根据生成器的定义，下面这个代码框中的numbers代表了全体自然数。在这个基础之上，我们可以利用filter()快速生成全体偶数自然数：

In [ ]:
def zrs():
    n = 0
    while True:
        yield n
        n += 1

numbers = zrs()

In [ ]:
next(numbers)

In [ ]:
numbers = zrs()
even_numbers = filter(lambda x: x % 2 == 0, numbers)

In [ ]:
next(even_numbers)

**排序函数sorted()**

排序是编程过程中经常遇到的场景，在Python中，可以利用sorted()函数对可迭代对象进行排序。我们先看一个例子：

In [ ]:
sorted([-10,-39,26,12,0])

在上例中，sorted()对一个只包含数字的list进行排序，返回的结果也是一个list。我们看到，排序的依据是数值大小，默认的顺序是升序。如果list只包含字符串呢？

In [ ]:
sorted(['abc','cyber','desk','apple','ago','wonder','zod','green'])

单从上例来看，对于由多个字符串组成的序列，sorted()是按照第一个字符升序排列的（在这里顺序由字母表决定），如果第一个字符相同，则比较第二个字符，依此类推。如果字符串里有数字、特殊符号、大小写呢？

In [ ]:
sorted(['abc','cyber','Desk','-','%',' ','100','2','11','Wonder','green',])

上面的排序依旧是按第一个字符升序排列，再按第二个字符排列，依此类推。排序的依据是ASCII（American Standard Code for Information Interchange，美国信息交换标准代码）。关于这个编码我们之后会有专门章节介绍，具体包含的元素和序号请参考[ASCII标准表](https://baike.baidu.com/item/ASCII/309296?fr=aladdin#3)。可以从表中看到，大写字母是排在小写字母之前的。

如果sorted()只能完成上述两种排序，那就差点意思了。事实上，它也是一个高阶函数，可以把排序标准对应的函数传至key参数，来实现自定义排序。以按绝对值排序为例：

In [ ]:
sorted([-10,-39,26,12,0], key = abs)

在上面这个排序的中间过程中，sorted()函数先把abs函数作用于列表中的每一个数字，生成对应的绝对值，组合成一个新序列，再根据这个序列敲定最终排序。

再举一例，如果想要根据字符串长度，对字符串列表进行排序，则应当把len()函数传至key参数：

In [ ]:
sorted(['ddd','aa','g','','aaaa'],key = len)

如果想忽略大小写，按首字母排序，则可以传入str.lower，其中lower()我们之前已接触过，是str对象的方法，本质上也是个函数：

In [ ]:
sorted(['abc','Cyber','Desk','wonder','Green'], key = str.lower)

sorted()函数还有另一个参数reverse，是反向排序（降序）的开关：

In [ ]:
sorted(['abc','Cyber','Desk','wonder','Green'], key = str.lower, reverse = True)

匿名函数和sorted()是绝配。来看下面这个例子：

In [ ]:
list_a = [(1,2),(2,5),(3,1),(6,4),(1,3),(0,0),(5,1),(4,6)]

list_a由多个tuple组成，现在我们要根据每个tuple的第二个元素进行升序排列，利用匿名函数，则可以写成以下形式：

In [ ]:
sorted(list_a,key=lambda x:x[1])

`lambda x:x[1]`这个匿名函数作用于一个tuple时，相当于返回该tuple的第二个元素。

**问题**：对如下list_b进行升序排序，排序先依据第三个元素、再依据第二个元素，该如何实现？

In [ ]:
list_a = [(1,2,3),(2,5,2),(3,1,1),(6,4,5),(1,3,6),(0,0,7),(5,1,1),(4,6,2)]

In [ ]:
sorted(list_a, key = lambda x: (x[2],x[1]))

**举一反三**：如果对第三个元素进行升序，但对第二个元素降序排列，该如何操作？

**提示**：对于数字，控制升降序没必要利用reverse参数，可以用相反数。

**问题**：以dict_a为例，如何根据字典的key或value进行排序？

In [ ]:
dict_a = dict([('cmcc',10086),('unicom',10010),('chinanet',10000),('bestone',12580)])

In [ ]:
dict_a

**解析**：Chapter 2曾强调，字典尽管以一定的顺序呈现，但它本身是没有排序概念的。受上一个例子启发，如果我们能把字典的键值对放在一个个tuple里组成一个序列，那就可以利用匿名函数进行排序。dict.items()方法可以帮我们实现这个目的：

In [ ]:
list(dict_a.items())

In [ ]:
sorted(dict_a.items(),key=lambda x:x[1])

以上代码实现了根据字典的值进行排序。注意，sorted()函数返回的只能是list，而不是字典。

### 5.3.4 返回一个函数

在函数式编程的世界里，函数不仅可以作为另一个函数的输入参数，还可以作为输出的返回值。用以下例子说明：

In [ ]:
def lazy_sum(*args):
    def cumul_add():
        result = 0
        for n in args:
            result += n
        return result
    return cumul_add

In [ ]:
lazy_sum(1,2,3,4,5)

在lazy_sum()函数里，我们定义了一个累加函数cumul_add()，然后返回这个函数。当我们调用lazy_sum()函数时，返回的就是一个function对象。

要真正调用这个返回的函数，只需要在后面加括号：

In [ ]:
f = lazy_sum(1,2,3,4,5)
f()

怎么理解上述过程？在调用lazy_sum()函数时，1,2,3,4,5传至可变参数args里生成一个tuple（关于可变参数，请回顾Chapter 4的内容，而在定义内部函数cumul_add()时，使用了这个tuple。所以，返回的函数实际上保存了这个tuple，作为参与内部运算的对象。

这样的设计结构，称为**闭包**（Closure）。

-----
## Part 1 小结
本章Part 1的内容初学者未必需要掌握，下列知识点在心里有个概念就差不多了：
- 迭代器的概念
- 迭代器对内存友好的特性
- 利用函数的形式定义生成器
- 函数可以作为另一个函数的输入参数或输出的返回值

但本章介绍的部分函数还是挺常见的，熟练使用它们能让代码变Pythonic：
- 匿名函数及其与常用高阶函数的组合拳
- 利用enumerate()、zip()简化遍历
- 利用map()、filter()操作可迭代对象的所有元素
- 利用sorted()对序列进行排序
- any()和all()

没有这些东西，不妨碍咱写代码，只不过会更冗长一些。如果你确实看完了这些内容后，在实操的时候就应当思考能不能偶尔利用进阶知识来几波“神操作”，这会让你的创造过程更有趣。

我们早就接触了模块（Module）这个概念，也从不同的模块中导入过一些函数和数据类型。Chapter 1总结Why Python时强调了其可扩展性，而扩展功能来自外部模块。Part 2将介绍模块和包（Package）的概念、安装和使用。学完之后你就可以调用其他人写好的复杂函数了。